In [1]:
cd D:\LAB\ADR\AgenticAdr

D:\LAB\ADR\AgenticAdr


In [2]:
import tempfile
import os
import re
import shutil
import subprocess
import google.generativeai as genai
from dotenv import load_dotenv
from openai import OpenAI

# --- Configuration ---
load_dotenv()
GOOGLE_API_KEY = os.getenv("GOOGLE")
OPENAI_API_KEY = os.getenv("OPENAI")

genai.configure(api_key=GOOGLE_API_KEY)
client = OpenAI(api_key=OPENAI_API_KEY)

In [3]:
# --- Helper Functions for Local Repo Handling ---

def call_llm(prompt: str, model_name: str = "gemini-2.5-pro") -> str:
    if model_name == "gemini-2.5-pro":
        model = genai.GenerativeModel(model_name)
        response = model.generate_content(prompt)
        return response.text 
    elif model_name == "gpt-5":
        response = client.chat.completions.create(model="gpt-5", messages=[{"role": "user", "content": prompt}]) 
        return response.choices[0].message.content


def clone_repo(repo_url: str) -> str:
    """
    Clones a public repository into a temporary local directory.
    Returns the path to the directory if successful, otherwise None.
    """
    temp_dir = tempfile.mkdtemp()
        
    print(f"Cloning {repo_url} into '{temp_dir}'...")
    try:
        # We use --depth 1 to do a shallow clone, which is faster and uses less space
        subprocess.check_call(
            ['git', 'clone', '--depth', '1', repo_url, temp_dir],
            stdout=subprocess.DEVNULL,
            stderr=subprocess.DEVNULL
        )
        print("✅ Cloning successful.")
        return temp_dir
    except (subprocess.CalledProcessError, FileNotFoundError) as e:
        print(f"🔥 Error cloning repository: {e}. Please check the URL and that Git is installed.")
        # Clean up the failed clone attempt
        if os.path.exists(temp_dir):
            shutil.rmtree(temp_dir)
        return None

def analyze_local_repo(repo_path: str) -> dict:
    """Analyzes a local repository to extract file structure and manifest contents."""
    print("🌳 Analyzing local repository structure...")
    file_paths = []
    manifest_content = ""
    
    key_files = [
        'package.json', 'requirements.txt', 'pom.xml', 'build.gradle',
        'Dockerfile', 'docker-compose.yml', 'go.mod', 'Gemfile', 'composer.json'
    ]
    
    for root, _, files in os.walk(repo_path):
        if '.git' in root:
            continue
        for name in files:
            relative_path = os.path.relpath(os.path.join(root, name), repo_path)
            file_paths.append(relative_path.replace("\\", "/"))
            if name in key_files:
                print(f"   - Found key file: {relative_path}")
                try:
                    with open(os.path.join(root, name), 'r', encoding='utf-8', errors='ignore') as f:
                        content = f.read()
                        manifest_content += f"\n\n--- Content of {name} ---\n{content[:2000]}\n"
                except Exception as e:
                    print(f"   - Warning: Could not read {name}: {e}")

    return {"file_paths": file_paths, "manifest_content": manifest_content}

def slugify(text: str) -> str:
    """
    Converts a string into a URL-friendly/filename-friendly slug.
    Example: "ADR-001: Use Docker for Containerization" -> "adr-001-use-docker-for-containerization"
    """
    text = text.lower()
    text = re.sub(r'[^a-z0-9\s-]', '', text)  # Remove invalid chars
    text = re.sub(r'[\s-]+', '-', text).strip('-') # Replace spaces and repeated hyphens
    return text

def save_adrs(adr_content: str, output_dir: str):
    """Parses the Gemini output and saves each ADR to a separate file."""
    print(f"\n💾 Saving ADRs to directory: '{output_dir}'")
    
    # Create the output directory if it doesn't exist
    os.makedirs(output_dir, exist_ok=True)
    
    # Split the content by the standard markdown horizontal rule
    adrs = adr_content.split('---')
    
    saved_count = 0
    for adr_text in adrs:
        adr_text = adr_text.strip()
        if not adr_text:
            continue

        # Find the title line (e.g., "### ADR-001: Use Next.js Framework")
        title_match = re.search(r"###\s*(.*)", adr_text)
        if title_match:
            title = title_match.group(1).strip()
            filename = slugify(title) + ".md"
            filepath = os.path.join(output_dir, filename)
            
            try:
                # Re-add the separator for formatting, except for the first file
                content_to_save = adr_text if saved_count == 0 else "---\n" + adr_text
                with open(filepath, 'w', encoding='utf-8') as f:
                    f.write(content_to_save)
                print(f"   - Saved: {filename}")
                saved_count += 1
            except IOError as e:
                print(f"   - 🔥 Error saving file {filename}: {e}")
        else:
            print("   - ⚠️ Warning: Could not find a title for an ADR block. Skipping.")

    if saved_count == 0:
        print("🤷 No valid ADRs were found in the output to save.")
    else:
        print(f"\n✅ Successfully saved {saved_count} ADR file(s).")


# --- Main Generation Function ---

def generate_adrs_for_repo(repo_url: str, output_dir: str = "generated_adrs", model_name: str = "gemini-2.5-flash"):
    """
    Clones, analyzes, and saves generated ADRs for a repository.
    """
    temp_repo_path = None
    # 1. Clone the repository
    temp_repo_path = clone_repo(repo_url)
    if not temp_repo_path:
        return

    # 2. Analyze the local files
    repo_context = analyze_local_repo(temp_repo_path)
    
    # 3. Construct the Prompt for Gemini
    prompt = f"""
    You are an expert software architect. Your task is to analyze the provided context from a software repository and infer the key architectural decisions that were made.

    Here is the context from the repository:
    ---
    {repo_context}
    ---

    Based *only* on the context provided above, please generate a list of Architectural Decision Records (ADRs).

    Use the following Markdown template for each ADR, separating them with a '---' horizontal rule:

    ---

    ### ADR-001: [Title of Decision]

    **Status:** Inferred
    **Context:** [Provide a detailed explanation of the problem or the set of circumstances that led to this decision. What was the architectural challenge?]
    **Decision:** [Clearly state the decision that was made. What was the chosen solution or technology?]
    **Consequences:** [Describe the results of this decision. What are the positive and negative impacts? What trade-offs were made?]
    """
    # Note: A shortened prompt is shown here for brevity. 
    # The full prompt from the previous answer should be used.
    
    # 4. Call the Gemini API
    print(f"\n🧠 Sending context to {model_name} for analysis. This may take a moment...")
    # model = genai.GenerativeModel('gemini-2.5-flash')
    response = call_llm(prompt, model_name=model_name)
    
    # 5. Save the result to files
    save_adrs(response, output_dir)
    

## Main

In [4]:
def generate_adrs(repo_url_to_process):
    output_directory_for_adrs = 'Generated_ADRs/' + repo_url_to_process[19:].removesuffix('.git').replace('/', '_')

    # Gemini - dir1
    output_directory_for_adrs_gemini = output_directory_for_adrs + '/dir1'
    model_name = "gemini-2.5-pro"
    generate_adrs_for_repo(repo_url_to_process, output_directory_for_adrs_gemini, model_name=model_name)

    # gpt - dir2
    output_directory_for_adrs_gpt = output_directory_for_adrs + '/dir2'
    model_name = "gpt-5"
    generate_adrs_for_repo(repo_url_to_process, output_directory_for_adrs_gpt, model_name=model_name)

In [ ]:
"""
Repository URL
https://github.com/karthikv1392/cs6401_se.git"
https://github.com/sa4s-serc/HarmonE
https://github.com/srini1978/carbonQL
https://github.com/srini1978/AzureCognitiveSearchDemo
https://github.com/likhithkanigolla/LLMOps-Platform
https://github.com/likhithkanigolla/CAPS-IIITH
https://github.com/sa4s-serc/EdgeMLBalancer
https://github.com/sa4s-serc/switch
https://github.com/sa4s-serc/EcoMLS
https://github.com/sa4s-serc/AdaMLS
https://github.com/akhiha/juice-shop
https://gitlab.com/20a_akhila/amazon-clone
https://gitlab.com/20a_akhila/movie-database
https://gitlab.com/20a_akhila/silly_story_generator
https://github.com/sambuaneesh/why-py
https://github.com/montycloud/moya
https://github.com/stanfordnlp/dspy
https://github.com/miryala10sathvika/Food-delivery
https://github.com/miryala10sathvika/Drawingeditor
https://github.com/legend479/HARPP
https://github.com/divyanash911/RSS-Reader
https://github.com/divyanash911/design-for-social-innovation-project
https://github.com/OSDG-IIITH/CabMiloge
https://github.com/sudokara/monkeytpe-contest
https://github.com/sudokara/bracket
https://github.com/sudokara/SpecFlow
https://github.com/microsoft/sarathi-serve
https://github.com/Poorvi-HC/DIP-Project
https://github.com/adyanshkakran/RL_project
https://github.com/adyanshkakran/Shravana
https://github.com/sudokara/SE_Project_3
https://github.com/adyanshkakran/MLOps
https://github.com/adyanshkakran/Reddit-clone
https://github.com/adyanshkakran/need-for-speed
https://github.com/Moya/Moya
https://github.com/sismics/music
https://github.com/ameykaran/esell/
"""

repos = ["https://github.com/ameykaran/esell"]


for repo_url_to_process in repos:
    generate_adrs(repo_url_to_process)

Cloning https://github.com/sismics/music into 'C:\Users\rudra\AppData\Local\Temp\tmpzriq9anb'...
✅ Cloning successful.
🌳 Analyzing local repository structure...
   - Found key file: pom.xml
   - Found key file: music-agent\pom.xml
   - Found key file: music-android\build.gradle
   - Found key file: music-android\app\build.gradle
   - Found key file: music-core\pom.xml
   - Found key file: music-distribution-debian\pom.xml
   - Found key file: music-distribution-docker\docker-compose.yml
   - Found key file: music-distribution-docker\Dockerfile
   - Found key file: music-distribution-mac\pom.xml
   - Found key file: music-distribution-redhat\pom.xml
   - Found key file: music-distribution-standalone\pom.xml
   - Found key file: music-distribution-windows\pom.xml
   - Found key file: music-web\pom.xml
   - Found key file: music-web\src\main\webapp\package.json
   - Found key file: music-web-common\pom.xml

🧠 Sending context to gemini-2.5-pro for analysis. This may take a moment...

💾 Sav